This is a notebook that I have created to test the various features of the BaconBits cape.

In order to easily use this notebook, I first need to get the PocketBeagle hooked up to the internet.

To do this, on my laptop, I type:
```
sudo su
iptables --table nat --append POSTROUTING --out-interface wlp3s0 -j MASQUERADE
iptables --append FORWARD --in-interface enx38d269566e5d -j ACCEPT
echo 1 > /proc/sys/net/ipv4/ip_forward
exit
```
and then on the PocketBeagle I type:
```
sudo su
ip route add default via 192.168.7.1
echo "nameserver 8.8.8.8" >> /etc/resolv.conf
exit
```

I then cloned the git repository using:
```
git clone https://github.com/b-trav/BaconBitsIntro.git
```

Initially when I set up git I did these commands to set my username, email, and to allow me to push without typing my github password:
```
git config --global user.name "Ben Travaglione"
git config --global user.email "ben@travaglione.com"
git config --global credential.helper store
```

## Potentiometer

Let's look at the potentiometer on the BaconBits.

In bash, we can grab the value of the potentiometer very easily:

In [3]:
f = '/sys/bus/iio/devices/iio\:device0/in_voltage0_raw'
!ls $f
!cat $f


/sys/bus/iio/devices/iio:device0/in_voltage0_raw
462


This can be a bit trickier in python because it does not seem to like the filename for some reason:

In [4]:
with open(f,'r') as pot:
    val = pot.readline()
    print(val)

IOError: [Errno 2] No such file or directory: '/sys/bus/iio/devices/iio\\:device0/in_voltage0_raw'

I can get around this problem by creating a symbolic link:

In [5]:
!ln -s /sys/bus/iio/devices/iio\:device0/in_voltage0_raw potentiometer

ln: failed to create symbolic link 'potentiometer': File exists


In [6]:
lnf = 'potentiometer'
with open(lnf,'r') as pot:
    val = pot.readline()
    print(val)

462



But this seems like a bit of a hack!